# 딥러닝 1일차 복습

이번 복습에서는 1일차에 학습한 핵심 개념들을 다시 정리하고, 실습 문제를 통해 이해도를 점검합니다.

## 학습 목표
- 손실함수, 최적화함수, 활성화함수의 개념 정리
- 정규화와 표준화의 차이점 이해
- CNN의 출력 크기 계산 능력 향상
- MLP와 CNN 모델 직접 구현


## 환경 설정

In [ ]:
# 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# 한글 폰트 설정
plt.rc("font", family="NanumGothic")
plt.rcParams["axes.unicode_minus"] = False

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)

# GPU 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

---
# Part 1: 객관식 퀴즈

먼저 1일차에 학습한 내용을 객관식 문제로 점검해봅시다.

## 퀴즈 1: 손실함수의 역할

**문제:** 딥러닝에서 손실함수(Loss Function)의 주요 역할은 무엇인가요?

1. 모델의 가중치를 초기화하는 역할
2. 모델의 예측값과 실제값의 차이를 수치화하는 역할
3. 활성화 함수를 선택하는 역할
4. 학습률을 자동으로 조정하는 역할

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- 손실함수는 모델의 **예측값(prediction)**과 **실제값(ground truth)** 간의 차이를 수치화합니다.
- 이 손실값(loss value)을 최소화하는 방향으로 모델을 학습시킵니다.
- 대표적인 손실함수:
  - 회귀: MSE (Mean Squared Error), MAE (Mean Absolute Error)
  - 분류: CrossEntropyLoss, BCELoss
- 손실함수의 값이 작을수록 모델의 성능이 좋다고 판단할 수 있습니다.

</details>

## 퀴즈 2: 경사하강법

**문제:** 경사하강법(Gradient Descent)에서 학습률(Learning Rate)이 너무 클 때 발생할 수 있는 문제는?

1. 학습 속도가 너무 느려진다
2. 최적값을 지나쳐서 발산할 수 있다
3. 가중치가 0으로 수렴한다
4. 손실함수가 항상 증가한다

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- 학습률이 너무 크면 경사를 따라 내려가는 **보폭이 너무 커져서** 최적값(minimum)을 지나칠 수 있습니다.
- 이 경우 손실값이 진동하거나 발산(diverge)할 수 있습니다.
- 반대로 학습률이 너무 작으면:
  - 학습 속도가 매우 느려집니다
  - Local minimum에 갇힐 가능성이 높아집니다
- 적절한 학습률 선택이 중요합니다 (보통 0.001 ~ 0.01 범위에서 시작)

</details>

## 퀴즈 3: 활성화 함수

**문제:** ReLU 활성화 함수의 수식은 무엇인가요?

1. $f(x) = \frac{1}{1 + e^{-x}}$
2. $f(x) = \max(0, x)$
3. $f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$
4. $f(x) = x \cdot \sigma(x)$

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- ReLU(Rectified Linear Unit)는 $f(x) = \max(0, x)$로 정의됩니다.
- 특징:
  - x > 0일 때: f(x) = x (선형)
  - x ≤ 0일 때: f(x) = 0
- 장점:
  - 계산이 간단하고 빠름
  - Gradient vanishing 문제 완화
- 단점:
  - Dying ReLU 문제 (음수 영역에서 gradient가 0)
- 보기의 다른 함수들:
  - 1번: Sigmoid
  - 3번: Tanh
  - 4번: Swish

</details>

## 퀴즈 4: CNN의 특징

**문제:** CNN(Convolutional Neural Network)에서 Pooling 레이어의 주요 목적은?

1. 파라미터 수를 증가시켜 표현력을 높인다
2. 특징 맵의 크기를 줄이고 중요한 특징을 추출한다
3. 가중치를 학습 가능하게 만든다
4. 과적합을 발생시켜 모델을 복잡하게 만든다

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- Pooling 레이어는 특징 맵(feature map)의 **공간적 크기를 줄이는** 역할을 합니다.
- 주요 목적:
  - **다운샘플링**: 특징 맵 크기 축소 → 계산량 감소
  - **중요 특징 추출**: 가장 두드러진 특징을 보존
  - **Translation Invariance**: 약간의 위치 변화에 강건함
  - **과적합 방지**: 파라미터 수 감소
- 종류:
  - Max Pooling: 최댓값 선택 (가장 많이 사용)
  - Average Pooling: 평균값 계산
- 중요: Pooling 레이어는 **학습 가능한 파라미터가 없습니다**

</details>

---
# Part 2: 서술형 문제 (용어 설명)

핵심 개념들을 자신의 언어로 설명해보세요.

## 문제 1: 손실함수 (Loss Function)

**문제:** 손실함수(Loss Function)가 무엇인지 설명하고, 회귀와 분류 문제에서 각각 어떤 손실함수를 주로 사용하는지 서술하세요.

---

### 여러분의 답안을 작성해보세요

```
[여기에 답안을 작성하세요]





```

<details>
<summary><b>힌트 보기</b></summary>

- 손실함수의 **목적**은 무엇인가요?
- 예측값과 실제값의 관계를 어떻게 표현하나요?
- 회귀는 연속적인 값을, 분류는 범주를 예측합니다
- MSE, MAE, CrossEntropy 등을 떠올려보세요

</details>

<details>
<summary><b>모범 답안 보기</b></summary>

### 모범 답안

**손실함수(Loss Function)**는 모델의 예측값과 실제 정답 간의 차이를 수치화하여 모델의 성능을 평가하는 함수입니다. 손실함수의 값이 작을수록 모델이 더 정확한 예측을 하고 있다는 의미이며, 학습 과정에서 이 손실값을 최소화하는 방향으로 모델의 가중치를 업데이트합니다.

**회귀 문제**에서는 주로 다음의 손실함수를 사용합니다:
- **MSE (Mean Squared Error)**: 예측값과 실제값의 차이를 제곱하여 평균을 구합니다. 큰 오차에 더 큰 패널티를 부여합니다.
- **MAE (Mean Absolute Error)**: 예측값과 실제값의 절댓값 차이의 평균입니다. 이상치에 MSE보다 덜 민감합니다.

**분류 문제**에서는 주로 다음의 손실함수를 사용합니다:
- **CrossEntropyLoss**: 다중 클래스 분류에 사용하며, 예측 확률 분포와 실제 레이블 간의 교차 엔트로피를 계산합니다.
- **BCELoss (Binary Cross Entropy)**: 이진 분류에 사용합니다.

**핵심 키워드:**
- 예측값과 실제값의 차이 수치화
- 모델 성능 평가
- 회귀: MSE, MAE
- 분류: CrossEntropyLoss, BCELoss
- 손실값 최소화 → 학습

</details>

## 문제 2: 최적화함수 (Optimizer)

**문제:** 최적화함수(Optimizer)의 역할을 설명하고, SGD와 Adam의 차이점을 서술하세요.

---

### 여러분의 답안을 작성해보세요

```
[여기에 답안을 작성하세요]





```

<details>
<summary><b>힌트 보기</b></summary>

- 최적화함수는 **가중치 업데이트**와 관련이 있습니다
- 경사하강법(Gradient Descent)의 변형들입니다
- SGD는 가장 기본적인 방법입니다
- Adam은 **적응형 학습률**을 사용합니다
- Momentum의 개념을 생각해보세요

</details>

<details>
<summary><b>모범 답안 보기</b></summary>

### 모범 답안

**최적화함수(Optimizer)**는 손실함수의 값을 최소화하기 위해 모델의 가중치를 업데이트하는 알고리즘입니다. 역전파를 통해 계산된 그래디언트(gradient)를 사용하여 가중치를 조정하며, 이 과정을 반복하여 모델이 최적의 성능을 내도록 학습시킵니다.

**SGD (Stochastic Gradient Descent)**:
- 가장 기본적인 최적화 알고리즘입니다.
- 각 미니배치마다 그래디언트를 계산하여 가중치를 업데이트합니다.
- 수식: $\theta = \theta - \eta \cdot \nabla_\theta L$
- 장점: 단순하고 메모리 효율적
- 단점: 학습률이 고정되어 있고, 진동이 심할 수 있으며, 수렴 속도가 느릴 수 있습니다.

**Adam (Adaptive Moment Estimation)**:
- 적응형 학습률을 사용하는 고급 최적화 알고리즘입니다.
- Momentum과 RMSProp의 장점을 결합했습니다.
- 각 파라미터마다 **다른 학습률**을 적용합니다.
- 1차 모멘트(평균)와 2차 모멘트(분산)를 모두 추정합니다.
- 장점: 빠른 수렴, 안정적인 학습, 하이퍼파라미터 튜닝이 덜 필요
- 단점: 메모리 사용량이 SGD보다 많음

**주요 차이점**:
- SGD는 고정 학습률, Adam은 적응형 학습률
- Adam이 일반적으로 더 빠르고 안정적으로 수렴
- Adam은 각 파라미터별로 학습률을 자동 조정

**핵심 키워드:**
- 가중치 업데이트
- 손실함수 최소화
- 경사하강법
- SGD: 고정 학습률, 단순
- Adam: 적응형 학습률, Momentum + RMSProp
- 그래디언트 기반 업데이트

</details>

## 문제 3: 활성화함수 (Activation Function)

**문제:** 활성화함수(Activation Function)의 필요성과 역할을 설명하고, ReLU, Sigmoid, Tanh 중 하나를 선택하여 그 특징을 서술하세요.

---

### 여러분의 답안을 작성해보세요

```
[여기에 답안을 작성하세요]





```

<details>
<summary><b>힌트 보기</b></summary>

- 활성화함수가 없으면 어떻게 될까요? (선형성)
- **비선형성(non-linearity)**이 왜 중요한가요?
- 각 활성화함수의 수식과 출력 범위를 생각해보세요
- Gradient vanishing 문제를 고려해보세요

</details>

<details>
<summary><b>모범 답안 보기</b></summary>

### 모범 답안

**활성화함수(Activation Function)**는 신경망에 **비선형성(non-linearity)**을 추가하여 복잡한 패턴을 학습할 수 있게 만드는 함수입니다.

**필요성:**
- 활성화함수가 없으면 여러 층을 쌓아도 결국 **선형 변환의 조합**이 되어 단일 선형 레이어와 동일합니다.
- 비선형 활성화함수를 사용해야 신경망이 XOR 문제와 같은 비선형 문제를 해결할 수 있습니다.
- 각 뉴런의 출력을 조절하여 다음 층으로 전달할 정보를 결정합니다.

**ReLU (Rectified Linear Unit)**의 특징:
- 수식: $f(x) = \max(0, x)$
- 출력 범위: $[0, \infty)$
- **장점:**
  - 계산이 매우 간단하고 빠름
  - Gradient vanishing 문제를 완화 (양수 영역에서 gradient가 1)
  - 희소성(sparsity) 제공 (음수는 0으로)
  - 실제로 가장 많이 사용되는 활성화함수
- **단점:**
  - Dying ReLU 문제: 음수 입력에 대해 gradient가 0이 되어 뉴런이 죽을 수 있음
  - 출력이 0 이상으로 제한되어 평균이 0이 아님
- **변형:** Leaky ReLU, PReLU, ELU 등이 Dying ReLU 문제를 해결하기 위해 제안됨

**핵심 키워드:**
- 비선형성 추가
- 복잡한 패턴 학습
- 선형 변환의 한계 극복
- ReLU: max(0, x), 계산 효율적, Gradient vanishing 완화
- Dying ReLU 문제
- 가장 널리 사용되는 활성화함수

</details>

## 문제 4: 정규화와 표준화

**문제:** 정규화(Normalization)와 표준화(Standardization)의 차이점을 설명하고, 각각 어떤 상황에서 사용하는 것이 적합한지 서술하세요.

---

### 여러분의 답안을 작성해보세요

```
[여기에 답안을 작성하세요]





```

<details>
<summary><b>힌트 보기</b></summary>

- 정규화: 데이터를 **특정 범위**로 변환 (보통 0~1)
- 표준화: **평균과 표준편차**를 사용하여 변환
- Min-Max Scaling vs Z-score Normalization
- 이상치(outlier)의 영향을 고려해보세요
- 각 방법의 수식을 떠올려보세요

</details>

<details>
<summary><b>모범 답안 보기</b></summary>

### 모범 답안

**정규화(Normalization)**와 **표준화(Standardization)**는 모두 데이터의 스케일을 조정하여 학습을 안정화하고 수렴 속도를 높이는 전처리 기법입니다.

**정규화 (Normalization, Min-Max Scaling)**:
- 데이터를 **특정 범위**(주로 [0, 1] 또는 [-1, 1])로 변환합니다.
- 수식: $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$
- **특징:**
  - 모든 값이 동일한 범위 내에 위치
  - 데이터의 분포 형태는 변하지 않음
  - 이상치에 민감함 (min, max가 이상치의 영향을 받음)
- **사용 시기:**
  - 데이터가 특정 범위에 제한되어야 할 때
  - 이미지 데이터 (픽셀값 0~255 → 0~1)
  - 신경망의 출력층에서 확률값이 필요할 때
  - 이상치가 적고 균등 분포에 가까운 데이터

**표준화 (Standardization, Z-score Normalization)**:
- 데이터를 **평균 0, 표준편차 1**인 분포로 변환합니다.
- 수식: $x' = \frac{x - \mu}{\sigma}$
- **특징:**
  - 데이터를 정규분포 형태로 변환
  - 이상치의 영향을 상대적으로 덜 받음
  - 데이터가 특정 범위로 제한되지 않음
- **사용 시기:**
  - 데이터가 정규분포를 따르거나 정규분포에 가까울 때
  - 이상치가 많은 데이터
  - 대부분의 머신러닝 알고리즘 (SVM, 선형회귀 등)
  - 딥러닝에서 입력 데이터 전처리 (일반적으로 더 선호됨)

**차이점 요약:**
| 구분 | 정규화 | 표준화 |
|------|--------|--------|
| 범위 | [0, 1] 또는 [-1, 1] | 제한 없음 |
| 기준 | Min, Max | 평균, 표준편차 |
| 이상치 민감도 | 높음 | 낮음 |
| 분포 형태 | 유지 | 정규분포로 변환 |

**핵심 키워드:**
- 정규화: Min-Max Scaling, [0, 1] 범위, 이상치 민감
- 표준화: Z-score, 평균 0 표준편차 1, 이상치 강건
- 데이터 스케일 조정
- 학습 안정화 및 수렴 속도 향상
- 이미지는 정규화, 일반 데이터는 표준화

</details>

---
# Part 3: CNN 필터 크기 계산 문제

CNN에서 출력 크기를 계산하는 능력은 매우 중요합니다. 다양한 상황에서 출력 크기를 계산해봅시다.

## CNN 출력 크기 계산 공식

**출력 크기 = $\lfloor \frac{입력크기 - 커널크기 + 2 \times padding}{stride} \rfloor + 1$**

- **입력 크기** (Input size): 입력 feature map의 가로/세로 크기
- **커널 크기** (Kernel size): 필터의 가로/세로 크기
- **Padding**: 입력 주변에 추가되는 0의 개수
- **Stride**: 필터가 이동하는 간격
- $\lfloor \cdot \rfloor$: 내림 (floor) 연산

## 문제 1: 기본 계산

**입력:** 32×32 이미지
**필터:** 5×5, stride=1, padding=0

**출력 크기는?**

---

### 여러분의 답안:
```
[여기에 계산 과정과 답을 작성하세요]


```

<details>
<summary><b>정답 보기</b></summary>

### 계산 과정:

공식: $\lfloor \frac{32 - 5 + 2 \times 0}{1} \rfloor + 1$

= $\lfloor \frac{32 - 5 + 0}{1} \rfloor + 1$

= $\lfloor \frac{27}{1} \rfloor + 1$

= $27 + 1$

= **28**

### 정답: 28×28

</details>

## 문제 2: Same Padding

**입력:** 28×28 이미지
**필터:** 3×3, stride=1, **same padding** (출력 크기가 입력과 같도록)

**필요한 padding 크기와 출력 크기는?**

---

### 여러분의 답안:
```
[여기에 계산 과정과 답을 작성하세요]


```

<details>
<summary><b>힌트 보기</b></summary>

- Same padding은 출력 크기 = 입력 크기가 되도록 padding을 설정하는 것입니다.
- stride=1일 때, padding = (kernel_size - 1) / 2 공식을 사용할 수 있습니다.
- 3×3 커널이면 padding은?

</details>

<details>
<summary><b>정답 보기</b></summary>

### 계산 과정:

**Same padding 공식 (stride=1):**

$padding = \frac{kernel\_size - 1}{2} = \frac{3 - 1}{2} = 1$

**출력 크기 계산:**

$\lfloor \frac{28 - 3 + 2 \times 1}{1} \rfloor + 1$

= $\lfloor \frac{28 - 3 + 2}{1} \rfloor + 1$

= $\lfloor \frac{27}{1} \rfloor + 1$

= $28$

### 정답: padding=1, 출력 크기 28×28

</details>

## 문제 3: Stride가 있는 경우

**입력:** 64×64 이미지
**필터:** 3×3, **stride=2**, padding=1

**출력 크기는?**

---

### 여러분의 답안:
```
[여기에 계산 과정과 답을 작성하세요]


```

<details>
<summary><b>정답 보기</b></summary>

### 계산 과정:

$\lfloor \frac{64 - 3 + 2 \times 1}{2} \rfloor + 1$

= $\lfloor \frac{64 - 3 + 2}{2} \rfloor + 1$

= $\lfloor \frac{63}{2} \rfloor + 1$

= $\lfloor 31.5 \rfloor + 1$

= $31 + 1$

= **32**

### 정답: 32×32

**핵심:** Stride가 2이면 출력 크기가 대략 절반으로 줄어듭니다.

</details>

## 문제 4: Pooling과 결합

**입력:** 224×224 이미지
**연산 순서:**
1. Conv: 7×7, stride=2, padding=3
2. MaxPool: 3×3, stride=2, padding=1

**최종 출력 크기는?**

---

### 여러분의 답안:
```
[여기에 계산 과정과 답을 작성하세요]


```

<details>
<summary><b>정답 보기</b></summary>

### 계산 과정:

**1단계 - Conv 레이어 후:**

$\lfloor \frac{224 - 7 + 2 \times 3}{2} \rfloor + 1$

= $\lfloor \frac{224 - 7 + 6}{2} \rfloor + 1$

= $\lfloor \frac{223}{2} \rfloor + 1$

= $111 + 1 = 112$

**2단계 - MaxPool 레이어 후:**

$\lfloor \frac{112 - 3 + 2 \times 1}{2} \rfloor + 1$

= $\lfloor \frac{112 - 3 + 2}{2} \rfloor + 1$

= $\lfloor \frac{111}{2} \rfloor + 1$

= $55 + 1 = 56$

### 정답: 56×56

**참고:** 이는 ResNet의 초기 레이어와 유사한 구조입니다.

</details>

## 문제 5: 복합 계산 (도전 문제)

**입력:** 128×128×3 (RGB 이미지)
**연산 순서:**
1. Conv1: 3×3, 64 filters, stride=1, padding=1
2. Conv2: 3×3, 64 filters, stride=1, padding=1
3. MaxPool: 2×2, stride=2, padding=0
4. Conv3: 3×3, 128 filters, stride=1, padding=1
5. MaxPool: 2×2, stride=2, padding=0

**각 단계별 출력 크기 (H×W×C)를 구하세요.**

---

### 여러분의 답안:
```
[여기에 계산 과정과 답을 작성하세요]




```

<details>
<summary><b>힌트 보기</b></summary>

- Same padding (padding=1, stride=1)은 크기를 유지합니다
- MaxPool (2×2, stride=2)은 크기를 절반으로 줄입니다
- 채널 수는 필터의 개수로 결정됩니다
- 단계별로 차근차근 계산해보세요

</details>

<details>
<summary><b>정답 보기</b></summary>

### 계산 과정:

**입력:** 128×128×3

**1단계 - Conv1 (3×3, 64 filters, stride=1, padding=1):**
- H, W: $\lfloor \frac{128 - 3 + 2 \times 1}{1} \rfloor + 1 = \lfloor \frac{127}{1} \rfloor + 1 = 128$
- C: 64 (필터 개수)
- **출력: 128×128×64**

**2단계 - Conv2 (3×3, 64 filters, stride=1, padding=1):**
- H, W: 동일하게 128 유지
- C: 64
- **출력: 128×128×64**

**3단계 - MaxPool (2×2, stride=2, padding=0):**
- H, W: $\lfloor \frac{128 - 2 + 0}{2} \rfloor + 1 = \lfloor \frac{126}{2} \rfloor + 1 = 63 + 1 = 64$
- C: 64 (변화 없음)
- **출력: 64×64×64**

**4단계 - Conv3 (3×3, 128 filters, stride=1, padding=1):**
- H, W: 64 유지
- C: 128 (필터 개수)
- **출력: 64×64×128**

**5단계 - MaxPool (2×2, stride=2, padding=0):**
- H, W: $\lfloor \frac{64 - 2 + 0}{2} \rfloor + 1 = \lfloor \frac{62}{2} \rfloor + 1 = 31 + 1 = 32$
- C: 128 (변화 없음)
- **최종 출력: 32×32×128**

### 정답:
- Conv1 출력: 128×128×64
- Conv2 출력: 128×128×64
- MaxPool1 출력: 64×64×64
- Conv3 출력: 64×64×128
- MaxPool2 출력: **32×32×128**

</details>

In [ ]:
# CNN 출력 크기를 자동으로 계산해주는 함수
def calculate_conv_output_size(input_size, kernel_size, stride=1, padding=0):
    """
    Conv2d 레이어의 출력 크기를 계산합니다.

    Parameters:
    -----------
    input_size : int
        입력 feature map의 크기 (H 또는 W)
    kernel_size : int
        필터의 크기
    stride : int
        stride 값
    padding : int
        padding 값

    Returns:
    --------
    int : 출력 크기
    """
    output_size = ((input_size - kernel_size + 2 * padding) // stride) + 1
    return output_size


def calculate_pool_output_size(input_size, pool_size, stride=None, padding=0):
    """
    Pooling 레이어의 출력 크기를 계산합니다.

    Parameters:
    -----------
    input_size : int
        입력 feature map의 크기 (H 또는 W)
    pool_size : int
        pooling window 크기
    stride : int or None
        stride 값 (None이면 pool_size와 동일)
    padding : int
        padding 값

    Returns:
    --------
    int : 출력 크기
    """
    if stride is None:
        stride = pool_size
    output_size = ((input_size - pool_size + 2 * padding) // stride) + 1
    return output_size


# 사용 예시
print("=== CNN 출력 크기 계산 도구 ===\n")

# 예시 1: 기본 계산
input_size = 32
output = calculate_conv_output_size(input_size, kernel_size=5, stride=1, padding=0)
print(f"문제 1 검증: {input_size}×{input_size} → {output}×{output}")

# 예시 2: Same padding
input_size = 28
output = calculate_conv_output_size(input_size, kernel_size=3, stride=1, padding=1)
print(f"문제 2 검증: {input_size}×{input_size} → {output}×{output}")

# 예시 3: Stride가 있는 경우
input_size = 64
output = calculate_conv_output_size(input_size, kernel_size=3, stride=2, padding=1)
print(f"문제 3 검증: {input_size}×{input_size} → {output}×{output}")

# 예시 4: Pooling과 결합
input_size = 224
output1 = calculate_conv_output_size(input_size, kernel_size=7, stride=2, padding=3)
output2 = calculate_pool_output_size(output1, pool_size=3, stride=2, padding=1)
print(
    f"문제 4 검증: {input_size}×{input_size} → {output1}×{output1} → {output2}×{output2}"
)

---
# Part 4: 코드 구현 문제

## 문제 1: 가중합 → 활성화 함수 → 출력 구현

**문제:** 신경망의 기본 동작인 "가중합 계산 → 활성화 함수 적용" 과정을 NumPy로 구현하세요.

**요구사항:**
1. 입력 벡터 x와 가중치 행렬 W, 편향 벡터 b를 받아 가중합(z = W·x + b)을 계산
2. ReLU, Sigmoid, Tanh 중 하나의 활성화 함수를 선택하여 적용
3. 최종 출력값을 반환

**입력 예시:**
- x = [1.0, 2.0, 3.0]
- W = [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]
- b = [0.1, 0.2]
- activation = 'relu'

In [ ]:
# TODO: 여러분의 코드를 작성하세요


def forward_pass(x, W, b, activation="relu"):
    """
    가중합 계산 후 활성화 함수를 적용합니다.

    Parameters:
    -----------
    x : array-like
        입력 벡터 (shape: (n,))
    W : array-like
        가중치 행렬 (shape: (m, n))
    b : array-like
        편향 벡터 (shape: (m,))
    activation : str
        활성화 함수 ('relu', 'sigmoid', 'tanh')

    Returns:
    --------
    output : ndarray
        활성화 함수 적용 후 출력 (shape: (m,))
    """
    # 여기에 코드를 작성하세요
    pass


# 테스트 코드
if __name__ == "__main__":
    # 예시 입력
    x = np.array([1.0, 2.0, 3.0])
    W = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
    b = np.array([0.1, 0.2])

    # 각 활성화 함수로 테스트
    for act in ["relu", "sigmoid", "tanh"]:
        result = forward_pass(x, W, b, activation=act)
        print(f"{act.upper()} 출력: {result}")

<details>
<summary><b>힌트 보기</b></summary>

### 힌트

**1단계: 가중합 계산**
```python
z = np.dot(W, x) + b  # 또는 W @ x + b
```

**2단계: 활성화 함수**
- ReLU: `np.maximum(0, z)`
- Sigmoid: `1 / (1 + np.exp(-z))`
- Tanh: `np.tanh(z)`

**3단계: 조건문으로 분기**
```python
if activation == 'relu':
    output = ...
elif activation == 'sigmoid':
    output = ...
```

</details>

<details>
<summary><b>모범 답안 및 해설 보기</b></summary>

### 모범 답안

</details>

In [ ]:
# 모범 답안 (접혀있음 - 위의 details 블록에서 확인)


def forward_pass_solution(x, W, b, activation="relu"):
    """
    가중합 계산 후 활성화 함수를 적용합니다.

    Parameters:
    -----------
    x : array-like
        입력 벡터 (shape: (n,))
    W : array-like
        가중치 행렬 (shape: (m, n))
    b : array-like
        편향 벡터 (shape: (m,))
    activation : str
        활성화 함수 ('relu', 'sigmoid', 'tanh')

    Returns:
    --------
    output : ndarray
        활성화 함수 적용 후 출력 (shape: (m,))
    """
    # 1. 가중합 계산: z = W·x + b
    z = np.dot(W, x) + b

    # 2. 활성화 함수 적용
    if activation == "relu":
        # ReLU: max(0, x)
        output = np.maximum(0, z)
    elif activation == "sigmoid":
        # Sigmoid: 1 / (1 + e^(-x))
        output = 1 / (1 + np.exp(-z))
    elif activation == "tanh":
        # Tanh: (e^x - e^(-x)) / (e^x + e^(-x))
        output = np.tanh(z)
    else:
        raise ValueError(f"지원하지 않는 활성화 함수: {activation}")

    return output


# 상세 테스트
print("=" * 60)
print("모범 답안 테스트")
print("=" * 60)

x = np.array([1.0, 2.0, 3.0])
W = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]])
b = np.array([0.1, 0.2])

print(f"\n입력 벡터 x: {x}")
print(f"가중치 행렬 W:\n{W}")
print(f"편향 벡터 b: {b}")

# 가중합 계산 (중간 단계 확인)
z = np.dot(W, x) + b
print(f"\n가중합 z = W·x + b: {z}")
print(f"  - W의 첫 번째 행: [0.1, 0.2, 0.3]")
print(f"  - 0.1*1 + 0.2*2 + 0.3*3 + 0.1 = {0.1 * 1 + 0.2 * 2 + 0.3 * 3 + 0.1}")
print(f"  - W의 두 번째 행: [0.4, 0.5, 0.6]")
print(f"  - 0.4*1 + 0.5*2 + 0.6*3 + 0.2 = {0.4 * 1 + 0.5 * 2 + 0.6 * 3 + 0.2}")

print("\n" + "=" * 60)
print("활성화 함수별 출력")
print("=" * 60)

for act in ["relu", "sigmoid", "tanh"]:
    result = forward_pass_solution(x, W, b, activation=act)
    print(f"\n{act.upper()}:")
    print(f"  출력: {result}")

    if act == "relu":
        print(
            f"  설명: max(0, {z[0]:.2f}) = {result[0]:.4f}, max(0, {z[1]:.2f}) = {result[1]:.4f}"
        )
    elif act == "sigmoid":
        print(
            f"  설명: 1/(1+e^(-{z[0]:.2f})) = {result[0]:.4f}, 1/(1+e^(-{z[1]:.2f})) = {result[1]:.4f}"
        )
    elif act == "tanh":
        print(
            f"  설명: tanh({z[0]:.2f}) = {result[0]:.4f}, tanh({z[1]:.2f}) = {result[1]:.4f}"
        )

### 해설

**핵심 개념:**
1. **가중합 계산**: 선형 변환을 수행합니다 (z = W·x + b)
2. **활성화 함수**: 비선형성을 추가하여 복잡한 패턴을 학습 가능하게 합니다
3. **forward pass**: 입력에서 출력으로 신호가 전달되는 과정

**각 활성화 함수의 특징:**
- **ReLU**: 계산이 빠르고 gradient vanishing 완화, 음수는 0으로
- **Sigmoid**: 0~1 사이 값, 확률 해석 가능, gradient vanishing 문제
- **Tanh**: -1~1 사이 값, Sigmoid보다 중심이 0에 가까워 학습에 유리

**실무 팁:**
- 은닉층: ReLU 계열 (ReLU, Leaky ReLU, ELU)
- 이진 분류 출력층: Sigmoid
- 다중 클래스 분류 출력층: Softmax
- 회귀 출력층: 활성화 함수 없음 (선형)

---
## 문제 2: 간단한 MLP 모델 구현 (MNIST)

**문제:** MNIST 손글씨 숫자 분류를 위한 Multi-Layer Perceptron (MLP) 모델을 구현하세요.

**모델 구조:**
```
Input (784) → FC1 (512, ReLU) → Dropout(0.2) → FC2 (256, ReLU) → Dropout(0.2) → Output (10)
```

**요구사항:**
1. PyTorch의 nn.Module을 상속받는 MLP 클래스 작성
2. `__init__` 메서드에서 레이어 정의
3. `forward` 메서드에서 순전파 구현
4. 제공된 베이스라인 코드를 완성하여 학습 및 평가

### 베이스라인 코드

In [ ]:
# MNIST 데이터 로드
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

train_dataset = datasets.MNIST("./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST("./data", train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"학습 데이터: {len(train_dataset)}개")
print(f"테스트 데이터: {len(test_dataset)}개")
print(f"이미지 크기: {train_dataset[0][0].shape}")

# 샘플 이미지 시각화
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = train_dataset[i]
    ax.imshow(image.squeeze(), cmap="gray")
    ax.set_title(f"Label: {label}")
    ax.axis("off")
plt.tight_layout()
plt.show()

### TODO: MLP 모델 구현

In [ ]:
# TODO: 여러분의 코드를 작성하세요


class SimpleMLP(nn.Module):
    """
    간단한 Multi-Layer Perceptron 모델

    Architecture:
    - Input: 784 (28x28 flattened)
    - FC1: 512 neurons + ReLU
    - Dropout: 0.2
    - FC2: 256 neurons + ReLU
    - Dropout: 0.2
    - Output: 10 classes
    """

    def __init__(self):
        super(SimpleMLP, self).__init__()
        # TODO: 레이어들을 정의하세요
        pass

    def forward(self, x):
        """
        순전파

        Parameters:
        -----------
        x : torch.Tensor
            입력 이미지 (batch_size, 1, 28, 28)

        Returns:
        --------
        output : torch.Tensor
            클래스별 로짓 (batch_size, 10)
        """
        # TODO: 순전파 과정을 구현하세요
        pass


# TODO: 학습 함수 구현
def train_model(model, train_loader, criterion, optimizer, device):
    """
    모델을 1 에폭 학습시킵니다.
    """
    # TODO: 학습 코드를 작성하세요
    pass


# TODO: 평가 함수 구현
def evaluate_model(model, test_loader, criterion, device):
    """
    모델을 평가합니다.
    """
    # TODO: 평가 코드를 작성하세요
    pass


# 모델 생성 및 학습
if __name__ == "__main__":
    model = SimpleMLP().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # TODO: 학습 루프 작성
    pass

<details>
<summary><b>힌트 보기</b></summary>

### 힌트

**모델 구조:**
```python
self.fc1 = nn.Linear(784, 512)
self.dropout1 = nn.Dropout(0.2)
self.fc2 = nn.Linear(512, 256)
self.dropout2 = nn.Dropout(0.2)
self.fc3 = nn.Linear(256, 10)
```

**Forward 함수:**
```python
x = x.view(x.size(0), -1)  # Flatten
x = F.relu(self.fc1(x))
x = self.dropout1(x)
# ... 계속
```

**학습 루프:**
```python
model.train()
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
```

</details>

<details>
<summary><b>모범 답안 및 해설 보기</b></summary>

### 모범 답안

</details>

In [ ]:
# 모범 답안


class SimpleMLP_Solution(nn.Module):
    """
    간단한 Multi-Layer Perceptron 모델

    Architecture:
    - Input: 784 (28x28 flattened)
    - FC1: 512 neurons + ReLU
    - Dropout: 0.2
    - FC2: 256 neurons + ReLU
    - Dropout: 0.2
    - Output: 10 classes
    """

    def __init__(self):
        super(SimpleMLP_Solution, self).__init__()
        # 레이어 정의
        self.fc1 = nn.Linear(28 * 28, 512)  # 입력층 → 은닉층1
        self.dropout1 = nn.Dropout(0.2)  # Dropout 1
        self.fc2 = nn.Linear(512, 256)  # 은닉층1 → 은닉층2
        self.dropout2 = nn.Dropout(0.2)  # Dropout 2
        self.fc3 = nn.Linear(256, 10)  # 은닉층2 → 출력층

    def forward(self, x):
        """
        순전파

        Parameters:
        -----------
        x : torch.Tensor
            입력 이미지 (batch_size, 1, 28, 28)

        Returns:
        --------
        output : torch.Tensor
            클래스별 로짓 (batch_size, 10)
        """
        # 1. 입력을 1차원으로 평탄화 (Flatten)
        # (batch_size, 1, 28, 28) → (batch_size, 784)
        x = x.view(x.size(0), -1)

        # 2. 첫 번째 은닉층 + ReLU + Dropout
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        # 3. 두 번째 은닉층 + ReLU + Dropout
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        # 4. 출력층 (활성화 함수 없음, CrossEntropyLoss가 softmax 포함)
        x = self.fc3(x)

        return x


def train_model_solution(model, train_loader, criterion, optimizer, device):
    """
    모델을 1 에폭 학습시킵니다.

    Returns:
    --------
    avg_loss : float
        평균 손실값
    accuracy : float
        정확도 (%)
    """
    model.train()  # 학습 모드

    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # 1. 데이터를 디바이스로 이동
        images = images.to(device)
        labels = labels.to(device)

        # 2. 그래디언트 초기화
        optimizer.zero_grad()

        # 3. 순전파
        outputs = model(images)

        # 4. 손실 계산
        loss = criterion(outputs, labels)

        # 5. 역전파
        loss.backward()

        # 6. 가중치 업데이트
        optimizer.step()

        # 7. 통계 업데이트
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy


def evaluate_model_solution(model, test_loader, criterion, device):
    """
    모델을 평가합니다.

    Returns:
    --------
    avg_loss : float
        평균 손실값
    accuracy : float
        정확도 (%)
    """
    model.eval()  # 평가 모드

    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # 그래디언트 계산 비활성화
        for images, labels in test_loader:
            # 1. 데이터를 디바이스로 이동
            images = images.to(device)
            labels = labels.to(device)

            # 2. 순전파
            outputs = model(images)

            # 3. 손실 계산
            loss = criterion(outputs, labels)

            # 4. 통계 업데이트
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy


# 모델 학습 실행
print("=" * 60)
print("SimpleMLP 모델 학습")
print("=" * 60)

model = SimpleMLP_Solution().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 정보 출력
print(f"\n모델 구조:")
print(model)
print(f"\n총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

# 학습
num_epochs = 5
train_losses = []
train_accs = []
test_losses = []
test_accs = []

print(f"\n학습 시작 (총 {num_epochs} 에폭)")
print("-" * 60)

for epoch in range(num_epochs):
    # 학습
    train_loss, train_acc = train_model_solution(
        model, train_loader, criterion, optimizer, device
    )
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 평가
    test_loss, test_acc = evaluate_model_solution(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print("-" * 60)

# 학습 과정 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 손실 그래프
ax1.plot(range(1, num_epochs + 1), train_losses, "b-o", label="Train Loss")
ax1.plot(range(1, num_epochs + 1), test_losses, "r-o", label="Test Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("학습/테스트 손실")
ax1.legend()
ax1.grid(True)

# 정확도 그래프
ax2.plot(range(1, num_epochs + 1), train_accs, "b-o", label="Train Accuracy")
ax2.plot(range(1, num_epochs + 1), test_accs, "r-o", label="Test Accuracy")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy (%)")
ax2.set_title("학습/테스트 정확도")
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"\n최종 테스트 정확도: {test_accs[-1]:.2f}%")

### 예측 결과 시각화

In [ ]:
# 일부 테스트 이미지에 대한 예측 시각화
model.eval()

# 테스트 데이터에서 무작위로 샘플 추출
indices = np.random.choice(len(test_dataset), 10, replace=False)

fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for idx, ax in zip(indices, axes.flat):
    image, true_label = test_dataset[idx]

    # 예측
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(device)
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        pred_label = predicted.item()

    # 시각화
    ax.imshow(image.squeeze(), cmap="gray")
    color = "green" if pred_label == true_label else "red"
    ax.set_title(f"True: {true_label}, Pred: {pred_label}", color=color)
    ax.axis("off")

plt.suptitle("MLP 예측 결과 (초록=정답, 빨강=오답)", fontsize=14)
plt.tight_layout()
plt.show()

### 해설

**핵심 포인트:**

1. **모델 구조**
   - MLP는 완전 연결층(Fully Connected Layer)만으로 구성
   - 이미지를 1차원 벡터로 펼쳐서(flatten) 입력
   - 은닉층에는 ReLU 활성화 함수 사용
   - Dropout으로 과적합 방지

2. **학습 과정**
   - `model.train()`: Dropout 활성화
   - `optimizer.zero_grad()`: 그래디언트 초기화 (필수!)
   - `loss.backward()`: 역전파로 그래디언트 계산
   - `optimizer.step()`: 가중치 업데이트

3. **평가 과정**
   - `model.eval()`: Dropout 비활성화
   - `torch.no_grad()`: 그래디언트 계산 안 함 (메모리 절약)

4. **성능 분석**
   - MNIST는 비교적 쉬운 데이터셋
   - 단순 MLP로도 97~98% 정확도 달성 가능
   - CNN을 사용하면 99% 이상 가능

**주요 하이퍼파라미터:**
- 학습률(lr): 0.001 (Adam의 기본값)
- 배치 크기: 128
- Dropout 비율: 0.2
- 은닉층 크기: 512, 256

**개선 방법:**
- 더 깊은 네트워크 (레이어 추가)
- Batch Normalization 추가
- Learning rate scheduling
- Data augmentation

---
## 문제 3: 간단한 CNN 모델 구현 (MNIST)

**문제:** MNIST 손글씨 숫자 분류를 위한 Convolutional Neural Network (CNN) 모델을 구현하세요.

**모델 구조:**
```
Input (1×28×28)
→ Conv1 (32 filters, 3×3, padding=1) → ReLU → MaxPool (2×2)
→ Conv2 (64 filters, 3×3, padding=1) → ReLU → MaxPool (2×2)
→ Flatten
→ FC1 (128) → ReLU → Dropout(0.5)
→ FC2 (10)
```

**요구사항:**
1. PyTorch의 nn.Module을 상속받는 CNN 클래스 작성
2. Convolutional Layer와 Pooling Layer 사용
3. MLP와 성능 비교

### TODO: CNN 모델 구현

In [ ]:
# TODO: 여러분의 코드를 작성하세요


class SimpleCNN(nn.Module):
    """
    간단한 Convolutional Neural Network 모델

    Architecture:
    - Conv1: 32 filters, 3x3, padding=1
    - MaxPool: 2x2
    - Conv2: 64 filters, 3x3, padding=1
    - MaxPool: 2x2
    - FC1: 128 neurons
    - Dropout: 0.5
    - FC2: 10 classes
    """

    def __init__(self):
        super(SimpleCNN, self).__init__()
        # TODO: 레이어들을 정의하세요
        pass

    def forward(self, x):
        """
        순전파

        Parameters:
        -----------
        x : torch.Tensor
            입력 이미지 (batch_size, 1, 28, 28)

        Returns:
        --------
        output : torch.Tensor
            클래스별 로짓 (batch_size, 10)
        """
        # TODO: 순전파 과정을 구현하세요
        pass


# 모델 생성 및 학습
if __name__ == "__main__":
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # TODO: 학습 루프 작성 (MLP와 동일한 방식)
    pass

<details>
<summary><b>힌트 보기</b></summary>

### 힌트

**Convolutional 레이어:**
```python
self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
```

**출력 크기 계산:**
- 입력: 28×28
- Conv1 + Pool: 28×28 → 14×14 (32 channels)
- Conv2 + Pool: 14×14 → 7×7 (64 channels)
- Flatten: 7×7×64 = 3136

**Forward:**
```python
x = self.pool(F.relu(self.conv1(x)))
x = self.pool(F.relu(self.conv2(x)))
x = x.view(x.size(0), -1)  # Flatten
x = F.relu(self.fc1(x))
```

</details>

<details>
<summary><b>모범 답안 및 해설 보기</b></summary>

### 모범 답안

</details>

In [ ]:
# 모범 답안


class SimpleCNN_Solution(nn.Module):
    """
    간단한 Convolutional Neural Network 모델

    Architecture:
    - Conv1: 32 filters, 3x3, padding=1 → ReLU → MaxPool 2x2
    - Conv2: 64 filters, 3x3, padding=1 → ReLU → MaxPool 2x2
    - Flatten
    - FC1: 128 neurons → ReLU → Dropout 0.5
    - FC2: 10 classes
    """

    def __init__(self):
        super(SimpleCNN_Solution, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        # 입력 크기 계산: 28×28 → 14×14 → 7×7, 채널 64
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        """
        순전파

        Parameters:
        -----------
        x : torch.Tensor
            입력 이미지 (batch_size, 1, 28, 28)

        Returns:
        --------
        output : torch.Tensor
            클래스별 로짓 (batch_size, 10)
        """
        # Conv1 + ReLU + Pool
        # (batch, 1, 28, 28) → (batch, 32, 28, 28) → (batch, 32, 14, 14)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        # Conv2 + ReLU + Pool
        # (batch, 32, 14, 14) → (batch, 64, 14, 14) → (batch, 64, 7, 7)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        # Flatten
        # (batch, 64, 7, 7) → (batch, 64*7*7=3136)
        x = x.view(x.size(0), -1)

        # FC1 + ReLU + Dropout
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # FC2 (Output)
        x = self.fc2(x)

        return x


# CNN 모델 학습
print("=" * 60)
print("SimpleCNN 모델 학습")
print("=" * 60)

cnn_model = SimpleCNN_Solution().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# 모델 정보 출력
print(f"\n모델 구조:")
print(cnn_model)
print(f"\n총 파라미터 수: {sum(p.numel() for p in cnn_model.parameters()):,}")

# 학습
num_epochs = 5
cnn_train_losses = []
cnn_train_accs = []
cnn_test_losses = []
cnn_test_accs = []

print(f"\n학습 시작 (총 {num_epochs} 에폭)")
print("-" * 60)

for epoch in range(num_epochs):
    # 학습 (MLP와 동일한 함수 재사용)
    train_loss, train_acc = train_model_solution(
        cnn_model, train_loader, criterion, optimizer, device
    )
    cnn_train_losses.append(train_loss)
    cnn_train_accs.append(train_acc)

    # 평가
    test_loss, test_acc = evaluate_model_solution(
        cnn_model, test_loader, criterion, device
    )
    cnn_test_losses.append(test_loss)
    cnn_test_accs.append(test_acc)

    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print("-" * 60)

print(f"\n최종 테스트 정확도: {cnn_test_accs[-1]:.2f}%")

### MLP vs CNN 성능 비교

In [ ]:
# MLP vs CNN 비교 그래프
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 손실 비교
axes[0].plot(range(1, num_epochs + 1), train_losses, "b--", label="MLP Train")
axes[0].plot(range(1, num_epochs + 1), test_losses, "b-", label="MLP Test")
axes[0].plot(range(1, num_epochs + 1), cnn_train_losses, "r--", label="CNN Train")
axes[0].plot(range(1, num_epochs + 1), cnn_test_losses, "r-", label="CNN Test")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("MLP vs CNN - 손실 비교")
axes[0].legend()
axes[0].grid(True)

# 정확도 비교
axes[1].plot(range(1, num_epochs + 1), train_accs, "b--", label="MLP Train")
axes[1].plot(range(1, num_epochs + 1), test_accs, "b-", label="MLP Test")
axes[1].plot(range(1, num_epochs + 1), cnn_train_accs, "r--", label="CNN Train")
axes[1].plot(range(1, num_epochs + 1), cnn_test_accs, "r-", label="CNN Test")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy (%)")
axes[1].set_title("MLP vs CNN - 정확도 비교")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# 성능 비교 요약
print("\n" + "=" * 60)
print("MLP vs CNN 성능 비교 요약")
print("=" * 60)
print(f"MLP 최종 테스트 정확도: {test_accs[-1]:.2f}%")
print(f"CNN 최종 테스트 정확도: {cnn_test_accs[-1]:.2f}%")
print(f"정확도 향상: {cnn_test_accs[-1] - test_accs[-1]:.2f}%p")

### CNN 예측 결과 시각화 및 Feature Map 분석

In [ ]:
# CNN 예측 결과 시각화
cnn_model.eval()

fig, axes = plt.subplots(2, 5, figsize=(14, 6))
for idx, ax in zip(indices, axes.flat):
    image, true_label = test_dataset[idx]

    # 예측
    with torch.no_grad():
        image_tensor = image.unsqueeze(0).to(device)
        output = cnn_model(image_tensor)
        _, predicted = torch.max(output, 1)
        pred_label = predicted.item()

    # 시각화
    ax.imshow(image.squeeze(), cmap="gray")
    color = "green" if pred_label == true_label else "red"
    ax.set_title(f"True: {true_label}, Pred: {pred_label}", color=color)
    ax.axis("off")

plt.suptitle("CNN 예측 결과 (초록=정답, 빨강=오답)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Feature Map 시각화
def visualize_feature_maps(model, image, layer_name="conv1"):
    """
    CNN의 중간 feature map을 시각화합니다.
    """
    model.eval()

    # Hook을 사용하여 중간 레이어의 출력 캡처
    activation = {}

    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()

        return hook

    # Hook 등록
    if layer_name == "conv1":
        model.conv1.register_forward_hook(get_activation("conv1"))
    elif layer_name == "conv2":
        model.conv2.register_forward_hook(get_activation("conv2"))

    # Forward pass
    with torch.no_grad():
        _ = model(image.unsqueeze(0).to(device))

    # Feature map 추출
    feature_map = activation[layer_name].squeeze(0).cpu()

    return feature_map


# 샘플 이미지로 feature map 시각화
sample_image, sample_label = test_dataset[0]

# Conv1 feature maps
conv1_features = visualize_feature_maps(cnn_model, sample_image, "conv1")

fig, axes = plt.subplots(4, 8, figsize=(16, 8))
fig.suptitle(f"Conv1 Feature Maps (총 32개 필터)", fontsize=14)

for i, ax in enumerate(axes.flat):
    if i < conv1_features.shape[0]:
        ax.imshow(conv1_features[i], cmap="viridis")
        ax.set_title(f"Filter {i + 1}")
        ax.axis("off")
    else:
        ax.axis("off")

plt.tight_layout()
plt.show()

### 해설

**CNN vs MLP 핵심 차이점:**

1. **공간적 구조 보존**
   - MLP: 이미지를 1차원으로 펼침 → 공간 정보 손실
   - CNN: 2차원 구조 유지 → 지역적 패턴 학습

2. **파라미터 수**
   - MLP: 784×512 = 401,408개 (첫 레이어만)
   - CNN: 3×3×32 = 288개 (첫 레이어)
   - CNN이 훨씬 적은 파라미터로 더 좋은 성능

3. **특징 추출**
   - MLP: 픽셀별 가중치 학습
   - CNN: 엣지, 텍스처 등 계층적 특징 학습

4. **Translation Invariance**
   - CNN은 필터를 공유하므로 위치 변화에 강건

**CNN 구조 이해:**

```
입력: 1×28×28
   ↓
Conv1(32 filters, 3×3, pad=1): 32×28×28
   ↓ ReLU + MaxPool(2×2)
32×14×14
   ↓
Conv2(64 filters, 3×3, pad=1): 64×14×14
   ↓ ReLU + MaxPool(2×2)
64×7×7 = 3136
   ↓ Flatten
FC1(128)
   ↓ ReLU + Dropout
FC2(10)
```

**Feature Map 해석:**
- 첫 번째 층: 엣지, 방향성 등 저수준 특징 감지
- 두 번째 층: 코너, 곡선 등 조합된 특징
- 깊은 층으로 갈수록 추상적인 특징 학습

**성능 향상 팁:**
- Batch Normalization 추가
- 더 깊은 네트워크 (VGG, ResNet 스타일)
- Data Augmentation (회전, 이동, 확대/축소)
- Learning rate scheduling

**MNIST 데이터셋에서:**
- MLP: 약 97-98% 정확도
- 간단한 CNN: 약 99% 정확도
- 깊은 CNN (ResNet 등): 99.5% 이상